# Cross validation

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)
import pandas as pd
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
matplotlib.rcParams["figure.figsize"] = (15,4)
from src.symbols import *

# Read configs:
conf_path = os.path.join("../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmms = global_variables["vmms"]
only_joined = global_variables[
    "only_joined"
]  # (regress/predict with only models from joined runs)S

In [ ]:
from wPCC_pipeline.pipelines.motion_regression.nodes import fit_motions
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold

In [ ]:
join = "joined"
data = catalog.load(f"{ join }.data_ek_smooth")


added_masses = catalog.load("added_masses")
ship_data = catalog.load("ship_data")
#vmm = catalog.load("vmm_martins_simple")
vmm = catalog.load("vmm_abkowitz")
exclude_parameters = catalog.load("params:motion_regression.exclude_parameters")

In [ ]:
for key in ['u1d','v1d','r1d']:
    
    fig,ax=plt.subplots()
    fig.set_size_inches(10,5)
    data.plot(y=key, ax=ax)
    ax.set_title(key)
    

In [ ]:
regression, parameters = fit_motions(data=data, added_masses=added_masses, ship_data=ship_data, vmm=vmm, exclude_parameters=exclude_parameters)

In [ ]:
regression.model_X.summary()

In [ ]:
regression.parameters

In [ ]:
Xudot = added_masses['Xudot']

ship_data_prime = regression.ps.prime(ship_data)
m = ship_data_prime['m']

regression.parameters['regressed']/(-Xudot+m)


In [ ]:
regression.model_X.params['Xrr']*(-Xudot+m)
regression.model_X.bse['Xrr']*(-Xudot+m)

In [ ]:
X = data
random_state = 42
rkf = RepeatedKFold(n_splits=2, n_repeats=50, random_state=random_state)

df_parameters = pd.DataFrame()

for train, test in rkf.split(X):
    
    X_train = X.iloc[train]
       
    regression_, parameters = fit_motions(data=X_train,
                                         added_masses=added_masses,
                                         ship_data=ship_data,
                                         vmm=vmm,
                                         exclude_parameters=exclude_parameters)
    
    df_parameters=df_parameters.append(parameters["regressed"], ignore_index=True)
    

In [ ]:
df_parameters.mean()

In [ ]:
df_parameters.std()

In [ ]:
df_parameters.std() / df_parameters.mean()

In [ ]:
df_parameters.hist(column='Xrr', bins=20)

In [ ]:
import seaborn as sns
from scipy.stats import norm

#key = 'Xrr'
key = 'Xdeltadelta'
grid = sns.displot(df_parameters, x=key, kind="kde")

mu = regression.model_X.params[key]*(-Xudot+m)
std = regression.model_X.bse[key]*(-Xudot+m)

rv = norm(loc=mu, scale=std)
x = np.linspace(df_parameters[key].min(), df_parameters[key].max(),100)

ax = grid.ax
ax.plot(x,rv.pdf(x))

In [ ]:
df_parameters.plot()